In [2]:
from redisvl.extensions.cache.llm import SemanticCache
from redisvl.utils.vectorize import HFTextVectorizer

llmcache = SemanticCache(
    name="llmcache",  # underlying search index name
    redis_url="redis://localhost:6379",  # redis connection url string
    distance_threshold=0.05,  # semantic cache distance threshold
    vectorizer=HFTextVectorizer(
        "sentence-transformers/all-MiniLM-L6-v2"
    ),  # embdding model
    ttl=3600,
)

/home/ubuntu/dev/cora_customer_agent/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


22:17:43 sentence_transformers.SentenceTransformer INFO   Use pytorch device_name: cuda:0
22:17:43 sentence_transformers.SentenceTransformer INFO   Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
22:17:45 redisvl.index.index INFO   Index already exists, not overwriting.


In [ ]:
question = "What more about France?"

Cache hit!
Response: [{'entry_id': 'ecd58532a48b331b863f17bba72ebb7d5e82a128269f06a797ee1377266eb388', 'prompt': 'What more about France?', 'response': 'The capital of France is Paris.', 'vector_distance': 0.0, 'inserted_at': 1762721020.92, 'updated_at': 1762721020.92, 'key': 'llmcache:ecd58532a48b331b863f17bba72ebb7d5e82a128269f06a797ee1377266eb388'}]


In [19]:
response[0]["response"]

'The capital of France is Paris.'

In [3]:
response = await llmcache.acheck(question)
if response:
    print("Cache hit!")
    print("Response:", response)
else:
    print("Cache miss!")
    # Here you would normally call your LLM to get the response
    response = "The capital of France is Paris."
    llmcache.store(question, response)
    print("Stored response in cache.")

NameError: name 'question' is not defined